## Using `AutoGen` to Build Agentic AI with Tool Calling

Create a `requirements.txt` file with the following library.

```md
ag2
```

Create a `run.py` file with the following code.

```py
import datetime
import os
from typing import Annotated, Literal

from autogen import AssistantAgent, ConversableAgent, register_function
from autogen.coding import LocalCommandLineCodeExecutor

Operator = Literal["+", "-", "*", "/"]


# Define a custom function
def calculator(a: int, b: int, operator: Annotated[Operator, "operator"]) -> int:
    if operator == "+":
        return a + b
    elif operator == "-":
        return a - b
    elif operator == "*":
        return a * b
    elif operator == "/":
        return float(a / b)
    else:
        raise ValueError("Invalid operator")


# Define a helper function
def verify_config_func(u: str, k: str) -> bool:
    if u == "John" and k == "123":
        return True
    else:
        return False


# Define a helper function
def default_key_for_test() -> str:
    return "123"


# Setup agents

# method 1: use openai
# llm_config = {"model": "gpt-4-turbo", "api_key": api_key} # Replace with your actual model configuration

# method 2: use azure
AZURE_OPENAI_ENDPOINT = "xxx-prod-ai"
AZURE_OPENAI_API_VERSION = "2024-02-01"
AZURE_OPENAI_DEPLOYMENT_NAME = "xxx-GPT4-32K"
AZURE_OPENAI_DEPLOYMENT_NAME_EMBED = "xxx-TEXT-EMBEDDING-ADA"
AZURE_OPENAI_API_KEY = "xxx"
url = f"https://{AZURE_OPENAI_ENDPOINT}.openai.azure.com/openai/deployments/{AZURE_OPENAI_DEPLOYMENT_NAME}/chat/completions?api-version={AZURE_OPENAI_API_VERSION}"

llm_config = {
    "model": AZURE_OPENAI_DEPLOYMENT_NAME,
    "api_key": AZURE_OPENAI_API_KEY,
    "base_url": url,
    "api_type": "azure",
    "api_version": AZURE_OPENAI_API_VERSION,
}

# Let's first define the assistant agent that suggests tool calls.
assistant = ConversableAgent(
    name="Assistant",
    system_message="You are a helpful AI assistant. "
    "You always start by asking user for configuration. "
    "Please ask the user for information that can fill the values of the following dictionary. "
    "Template config file: {'user': 'John', 'key': '123'}. "
    "If the user can't remember or enters the wrong key, prompt to call the default key using 'default_key_for_test' function. "
    "Here you execute the function 'verify_config_func' first. "
    "If the return is True, you then execute the second function 'calculator'. "
    "You can help with simple calculations. "
    "Return 'TERMINATE' when the task is done.",
    llm_config=llm_config,
)

# The user proxy agent is used for interacting with the assistant agent
# and executes tool calls.
user_proxy = ConversableAgent(
    name="User",
    llm_config=False,
    is_termination_msg=lambda msg: msg.get("content") is not None
    and "TERMINATE" in msg["content"],
    human_input_mode="ALWAYS",
)


# Register the following function to the two agents.
register_function(
    calculator,
    caller=assistant,  # The assistant agent can suggest calls to the calculator.
    executor=user_proxy,  # The user proxy agent can execute the calculator calls.
    name="calculator",  # By default, the function name is used as the tool name.
    description="A simple calculator",  # A description of the tool.
)

# Register the following function to the two agents.
register_function(
    verify_config_func,
    caller=assistant,  # The assistant agent can suggest calls to the calculator.
    executor=user_proxy,  # The user proxy agent can execute the calculator calls.
    name="verify_user",  # By default, the function name is used as the tool name.
    description="A user verification call",  # A description of the tool.
)

# Register the following function to the two agents.
register_function(
    default_key_for_test,
    caller=assistant,  # The assistant agent can suggest calls to the calculator.
    executor=user_proxy,  # The user proxy agent can execute the calculator calls.
    name="default_key_for_test",  # By default, the function name is used as the tool name.
    description="A default key for test usage",  # A description of the tool.
)

# Let's get started!
initial_request_from_user = input("Enter request here: 👉 ")
chat_result = user_proxy.initiate_chat(
    assistant, message=initial_request_from_user
)


print("Congrats! The 'run.py' script just finished!")
```